In [ ]:
from torch.utils.data import Dataset, DataLoader
from glob import glob
import numpy as np
import astropy.io.fits as fits
import os
from tqdm.auto import tqdm
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from collections import defaultdict
import os
import copy
import json

In [ ]:
def compute_mean_std_from_histogram(bin_edges, bin_counts):
    # Calculate midpoints of each bin
    bin_midpoints = (bin_edges[:-1] + bin_edges[1:]) / 2
    
    # Calculate weighted sum and total count
    weighted_sum = np.sum(bin_midpoints * bin_counts)
    total_count = np.sum(bin_counts)
    
    # Calculate mean
    mean = weighted_sum / total_count
    
    # Calculate weighted squared deviation from mean
    weighted_squared_deviation = np.sum(bin_counts * (bin_midpoints - mean)**2)
    
    # Calculate weighted standard deviation
    std = np.sqrt(weighted_squared_deviation / total_count)
    
    return mean, std

def get_dataset_counts(dataset):
    
    bit_depth = ds[0].flatten()[0].nbytes
    n_vals = 2**(8*bit_depth)
    counts = np.zeros(n_vals, dtype=np.uint64)

    for d in tqdm(dataset):
        counts += np.bincount(d.flatten(), minlength=n_vals).astype(np.uint64)

    return counts
    
"""
counts = get_dataset_mean_std(ds)
hist_edges = np.arange(len(counts) + 1)
hist_counts = counts

# Compute mean and standard deviation from the histogram
mean, std = compute_mean_std_from_histogram(hist_edges, hist_counts)
"""

## Dataloader

In [ ]:
def extract_images_within_time_range(events, image_paths):
    selected_images = []

    # Organize image paths by date for efficient checking
    image_paths_by_date = {}
    for image_path in image_paths:
        image_date = datetime.strptime(os.path.basename(image_path).split('_')[0], '%Y%m%d').date()
        image_paths_by_date.setdefault(image_date, []).append(image_path)

    for event in events:

        if not event['visible']:
            continue

        if event['faint']:
            continue
        
        event_start_time = datetime.strptime(event['datetime'], '%Y/%m/%d %H:%M')
        event_stop_time = datetime.strptime(event['event_stop_time'], '%Y%m%d_%H%M%S')
        event_date = event_start_time.date()
        
        # Check images within the event's date
        if event_date in image_paths_by_date:

            paths = image_paths_by_date[event_date]

            next_day = event_date + timedelta(days=1)
            if next_day in image_paths_by_date:
                paths.extend(image_paths_by_date[next_day])
            
            for image_path in paths:
                image_timestamp = datetime.strptime('_'.join(os.path.basename(image_path).split('_')[:2]), '%Y%m%d_%H%M%S')
                if event_start_time <= image_timestamp <= event_stop_time:
                    selected_images.append(image_path)

    return selected_images

In [ ]:
class CMEDataset(Dataset):

    def __init__(self, root, events = [], pol='all', size=512):
        self.cache_dir = os.path.join(root, '.cache')
        os.makedirs(self.cache_dir, exist_ok=True)
        
        self.events = events
        self.stereo_a = glob(os.path.join(root, "201402*", "cor1", "stereo_a", "*", "*.fts"))
        self.stereo_b = glob(os.path.join(root, "201402*", "cor1", "stereo_b", "*", "*.fts"))
        self.pol = pol

        self.images = []

        if pol == 'all':
            self.mean = 2691.3037070368546 
            self.std = 2579.566574917962
            self.images.extend([im for im in self.stereo_a if os.path.basename(os.path.dirname(im)) != '1001.0'])
            self.images.extend([im for im in self.stereo_b if os.path.basename(os.path.dirname(im)) != '1001.0'])
        elif pol == 'sum':
            self.images.extend([im for im in self.stereo_a if 'n4' in im])
            self.images.extend([im for im in self.stereo_b if 'n4' in im])
            self.mean = 3658.224788149089
            self.std = 3399.0258091444553
        else:
            self.mean = 2691.3037070368546 
            self.std = 2579.566574917962

            self.images.extend([im for im in self.stereo_a if os.path.basename(os.path.dirname(im)) == pol])
            self.images.extend([im for im in self.stereo_b if os.path.basename(os.path.dirname(im)) == pol])

        # filter size
        for image in self.images:
            if fits.getdata(image).shape != (size,size):
                print(f"Removing {image}")
                self.images.remove(image)

        self.images = sorted(self.images)
        #self.images_for_date = defaultdict(lambda: defaultdict(list))
        self.images_for_date = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
        self.dates = set()
        
        for image_path in self.images:
            image_date = self._image_date(image_path)
            sat, angle = self._image_info(image_path)

            self.images_for_date[image_date][sat][angle].append(image_path)
            self.dates.add(image_date)
            
        self.transform = transforms.Compose([
            transforms.ToTensor(),                   # Convert image to PyTorch tensor
            transforms.Normalize(mean=self.mean, std=self.std) # Normalize using mean and std
        ])

        self._get_labels()
        self._gen_background()

    def _image_date(self, image):
        return datetime.strptime(os.path.basename(image).split('_')[0], '%Y%m%d').date()

    def _image_info(self, image_path):
        if 'stereo_a' in image_path: sat = 'stereo_a'
        elif 'stereo_b' in image_path: sat = 'stereo_b'
        if "0.0" in image_path: angle = 0
        elif "120.0" in image_path: angle = 120
        elif "240.0" in image_path: angle = 240
        elif "1001.0" in image_path: angle = 1001

        return sat, angle

    def _get_labels(self):
        self.positive_labels = set(extract_images_within_time_range(self.events['stereo_a'], self.stereo_a))
        self.positive_labels |= set(extract_images_within_time_range(self.events['stereo_b'], self.stereo_b))
        self.cme_images = set([im for im in self.images if im in self.positive_labels])

    def _gen_background(self, filter_cme=False):

        cache_root = os.path.join(self.cache_dir, "background")
        os.makedirs(cache_root, exist_ok=True)
        print("Cache:", cache_root)

        self.background = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
        
        #self.background = {}
        for date in tqdm(sorted(list(self.dates))):
            image_date = date - timedelta(days=1)
            date_string = date.strftime("%Y%m%d")
            
            #if date not in self.background:
            #    self.background[date] = {}
            
            if image_date not in self.images_for_date:
                image_date = date

            # TODO add polar angle if "all": sat, angle = self._image_info(image_path) to replace self.pol

            for sat in ['stereo_a', 'stereo_b']:
                cache_path = os.path.join(cache_root, f"{date_string}_{sat}_{self.pol}.npy")
    
                if os.path.exists(cache_path):
                    self.background[date][sat][self.pol] = np.load(cache_path)
                else:
                    imgs = self.images_for_date[image_date][sat][self.pol]

                    if filter_cme:
                        imgs = np.array([fits.getdata(im) for im in imgs if im not in self.cme_images])
                    else:
                        imgs = np.array([fits.getdata(im) for im in imgs])
                    
                    self.background[date][sat][self.pol] = np.median(imgs, axis=0)
                    np.save(cache_path, self.background[date][sat][self.pol])

    def _get_background(self, img_path):
        """
        given image path, retrieve correct background from dict
        TODO: expand to polar angle
        """
        image_date = self._image_date(image_path)
        sat, angle = self._image_info(image_path)
        bg = self.background[image_date][sat][angle]

        #if 'stereo_a' in image_path:
        #    bg = self.background[image_date]['stereo_a']
        #elif 'stereo_b' in image_path:
        #    bg = self.background[image_date]['stereo_b']
        #else:
        #    print("Error retrieving background for", img_path)

        return bg

    def _get_difference_image(self, i):
        """
        TODO: ensure differencing over same sat, same polar with i-1
        """
        # read raw images for current and previous
        # fits.getdata(ds.images_for_date[date]['stereo_b'][idx])
        raw_img_i = fits.getdata(self.images[i]).astype(np.float32)
        raw_img_j = fits.getdata(self.images[i-1]).astype(np.float32)

        # get background for current
        bg_i = self._get_background(self.images[i])

        # get difference image with subtracted background
        diff_img = (raw_img_i - bg_i) - (raw_img_j - bg_i)

        return diff_img

                                     
    def __getitem__(self, i):
        data  = self._get_difference_image(i)
        label = int(self.images[i] in self.cme_images)
        
        return self.transform(data), label

    def __len__(self):
        return len(self.images)

In [ ]:
with open('events_201402.json', 'r') as fp:
    events = json.load(fp)

#events['stereo_a'] = [events['stereo_a'][10]]
#events['stereo_b'] = [events['stereo_b'][10]]

#ds = CMEDataset(root="/media/josh/josh_tuf_a/data/fdl/2023/onboard", pol='0.0', events=events)
ds = CMEDataset(root="/mnt/onboard_data/classifier", pol='0.0', events=events)

In [ ]:
len(ds.cme_images)

In [ ]:
ds.images[:10]

In [ ]:
ds.images[-10:]

In [ ]:
len(ds.images)

In [ ]:
ds.images_for_date

In [ ]:
ds.dates

## Check mean std for normalisation

TODO:
* double check and update for when background is removed

In [ ]:
def compute_mean_std_from_histogram(bin_edges, bin_counts):
    # Calculate midpoints of each bin
    bin_midpoints = (bin_edges[:-1] + bin_edges[1:]) / 2
    
    # Calculate weighted sum and total count
    weighted_sum = np.sum(bin_midpoints * bin_counts)
    total_count = np.sum(bin_counts)
    
    # Calculate mean
    mean = weighted_sum / total_count
    
    # Calculate weighted squared deviation from mean
    weighted_squared_deviation = np.sum(bin_counts * (bin_midpoints - mean)**2)
    
    # Calculate weighted standard deviation
    std = np.sqrt(weighted_squared_deviation / total_count)
    
    return mean, std

def get_dataset_counts(dataset):
    
    #bit_depth = ds[0].flatten()[0].nbytes
    bit_depth = ds[0][0].flatten()[0].numpy().nbytes
    n_vals = 2**(8*bit_depth)
    counts = np.zeros(n_vals, dtype=np.uint64)

    for d in tqdm(dataset):
        #counts += np.bincount(d.flatten(), minlength=n_vals).astype(np.uint64)
        counts += np.bincount(d[0].flatten(), minlength=n_vals).astype(np.uint64)

    return counts

def get_dataloader_meanstd(ds):
    counts = get_dataset_counts(ds)
    hist_edges = np.arange(len(counts) + 1)
    hist_counts = counts

    # Compute mean and standard deviation from the histogram
    mean, std = compute_mean_std_from_histogram(hist_edges, hist_counts)

In [ ]:
%%time
ds = CMEDataset(root="/mnt/onboard_data/classifier", pol='0.0', events=events)


In [ ]:
ds.images[0]

In [ ]:
ds[0] # for now fails if background given as __get_item fails

In [ ]:
# must be wrong
get_dataloader_meanstd(ds)

## Check CME images

In [ ]:
date = datetime(2014, 2, 25).date()
bg = ds.background[date]['stereo_b']

plt.figure(figsize=(10,10))
plt.subplot(121)
plt.imshow(bg)
plt.colorbar(location='bottom')

plt.subplot(122)
_ = plt.hist(bg.flatten(), bins=256)

plt.tight_layout()

In [ ]:
date

In [ ]:
idx = 3
img_curr = fits.getdata(ds.images_for_date[date]['stereo_b'][idx])
img_prev = fits.getdata(ds.images_for_date[date]['stereo_b'][idx-1])

img = (img_curr - bg) - (img_prev - bg)

plt.figure(figsize=(10,10))
plt.subplot(121)
plt.imshow(img)
plt.colorbar(location='bottom')

plt.subplot(122)
_ = plt.hist(img.flatten(), bins=256)

plt.tight_layout()

In [ ]:
len(ds.images)

In [ ]:
from sunpy.map.maputils import all_coordinates_from_map
from sunpy.map import Map
from astropy.coordinates import SkyCoord
from matplotlib.colors import LogNorm, Normalize, PowerNorm
from rich.progress import Progress
from sunpy.map import Map
import astropy.units as u

def generate_image(img_fname, ref_map, vmin=0, vmax=20):
    """
    for single image
    """
    m = Map(img_fname)

    pixel_coords = all_coordinates_from_map(m)
    solar_center = SkyCoord(0 * u.deg, 0 * u.deg, frame=m.coordinate_frame)
    pixel_radii = np.sqrt(
        (pixel_coords.Tx - solar_center.Tx) ** 2
        + (pixel_coords.Ty - solar_center.Ty) ** 2
    )
    # r2 masking
    mask = 1 - ((pixel_radii / pixel_radii.max()) ** 2) * 0.5
    mask = mask.value
    mask[pixel_radii.value >= 0.9 * pixel_coords.Tx.max().value] = np.nan

    data = (m.data - ref_map) / mask
    
    # imshow is mirror to m.plot
    plt.imshow(data, origin="lower", cmap="stereocor2")

In [ ]:
#generate_image(ds.images_for_date[date][20], bg)